In [2]:
import pandas as pd
import os
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression

os.chdir('data/')

In [33]:
#on prend la liste des fichiers
files = [f for f in os.listdir('.') if os.path.isfile(f)]
data = pd.read_csv(files[0])
#la colonne +/- n'est pas prise en complte car on a pas sa moyenne pour les matchs de Wembenyama
data = data.loc[:, data.columns!='+/-']
#convertion des minutes par match en flottant
data.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in data.loc[:,'MP']]
#concatenation de toutes les tables csv en un seul tableau
for file in files[1:]:
    match = pd.read_csv(file)
    match = match.loc[:,match.columns!='+/-']
    match.loc[:,'MP'] = [float(x.split(':')[0])+ 0.006*float(x.split(':')[1]) for x in match.loc[:,'MP']]
    data = pd.concat([data,match])
data.reset_index(drop=True, inplace=True)

#remplace les données absentes par une données de la même colonne, avec la même valeur pour la variable Résultat
data_win = data[data['Résultat'] == 1]
data_lose = data[data['Résultat'] != 1]
for col in data_win.columns:
    for i,rows in data_win.iterrows():
        if pd.isna(data_win.loc[i,col]):
            data_win.loc[i,col] = np.random.choice(data_win[data_win[col].notna()].loc[:,col])
for col in data_lose.columns:
    for i,rows in data_lose.iterrows():
        if pd.isna(data_lose.loc[i,col]):
            data_lose.loc[i,col] = np.random.choice(data_lose[data_lose[col].notna()].loc[:,col])
data = pd.concat([data_win,data_lose])

X = data.loc[:, data.columns!='Résultat']
X = X.iloc[:,1:]
Y = data.loc[:,'Résultat']
VW = np.array([30.2,6.9,15.9,0.435,1.4,4.9,0.279,5.6,11.0,0.504,0.477,3.3,4.2,0.774,2.3,8.4,10.7,2.8,1.4,3.0,3.3,2.5,18.5])

In [34]:
#régression linéaire
reg = LinearRegression().fit(X, Y)
for k in range(len(reg)):
    print('coef de ',data.columns[k], ':', reg[k])

TypeError: object of type 'LinearRegression' has no len()

In [ ]:
#régression logistique
clf = LogisticRegression(random_state=0).fit(X, Y)